In [ ]:
# ============================================================================
# 1. IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÃO
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopandas as gpd
from pathlib import Path
import warnings
from datetime import datetime
import json

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuração de diretórios
BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SHAPEFILE_DIR = DATA_DIR / 'shapefiles'
OUTPUT_DIR = BASE_DIR / 'outputs' / 'figures'

# Cria diretório de saída
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📁 Diretório de saída: {OUTPUT_DIR.absolute()}")

# Configuração de plotly
import plotly.io as pio
pio.templates.default = "plotly_white"


## 2. CARREGAMENTO E PREPARAÇÃO DOS DADOS


In [ ]:
# Carrega dados consolidados
df_crimes = pd.read_csv(PROCESSED_DIR / 'crimes_consolidado.csv')
print(f"📊 Dados carregados: {len(df_crimes):,} registros")

# Carrega dados geoespaciais
gdf_crimes = gpd.read_file(PROCESSED_DIR / 'crimes_geo.geojson')
print(f"🗺️ Dados geoespaciais: {len(gdf_crimes):,} registros")

# Informações básicas dos dados
print("\n📋 INFORMAÇÕES BÁSICAS:")
print(f"Período: {df_crimes['ano'].min()} - {df_crimes['ano'].max()}")
print(f"Regiões: {df_crimes['regiao_administrativa'].nunique()}")
print(f"Tipos de crime: {df_crimes['tipo_crime'].nunique()}")
print(f"Colunas: {list(df_crimes.columns)}")

# Preview dos dados
print("\n📋 Preview dos dados:")
print(df_crimes.head())

# Estatísticas descritivas
print("\n📊 Estatísticas descritivas:")
print(df_crimes.describe())


In [ ]:
# Estatísticas por tipo de crime
stats_por_tipo = df_crimes.groupby('tipo_crime').agg({
    'total_ocorrencias': ['sum', 'mean', 'std', 'min', 'max'],
    'taxa_100k': ['mean', 'std', 'min', 'max']
}).round(2)

print("📊 ESTATÍSTICAS POR TIPO DE CRIME:")
print(stats_por_tipo)

# Gráfico de barras - total de ocorrências por tipo
fig_barras = px.bar(
    df_crimes.groupby('tipo_crime')['total_ocorrencias'].sum().reset_index(),
    x='tipo_crime',
    y='total_ocorrencias',
    title='Total de Ocorrências por Tipo de Crime',
    labels={'total_ocorrencias': 'Total de Ocorrências', 'tipo_crime': 'Tipo de Crime'}
)

fig_barras.update_layout(
    width=1000,
    height=600,
    xaxis_title="Tipo de Crime",
    yaxis_title="Total de Ocorrências",
    xaxis_tickangle=-45
)

fig_barras.show()

# Salva gráfico
fig_barras.write_html(OUTPUT_DIR / 'barras_tipo_crime.html')
print("✅ Gráfico de barras salvo")
